# Naive Bayes Tutorial Part 1: Predicting survival from titanic crash

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("titanic1.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

In [8]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [ ]:
I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [9]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [10]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [11]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [12]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [14]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [15]:
model.fit(X_train,y_train)

GaussianNB()

In [16]:
model.score(X_test,y_test)

0.7835820895522388

In [17]:
X_test[0:10]

,Pclass,Age,Fare,female
806,1,39.000000,0.0000,0
433,3,17.000000,7.1250,0
886,2,27.000000,13.0000,0
321,3,27.000000,7.8958,0
598,3,29.699118,7.2250,0
737,1,35.000000,512.3292,0
599,1,49.000000,56.9292,0
794,3,25.000000,7.8958,0
479,3,2.000000,12.2875,1
253,3,30.000000,16.1000,0


In [18]:
y_test[0:10]

806    0
433    0
886    0
321    0
598    0
737    1
599    1
794    0
479    1
253    0
Name: Survived, dtype: int64

In [19]:
model.predict(X_test[0:10])

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0], dtype=int64)

In [20]:
model.predict_proba(X_test[:10])

array([[7.48998592e-01, 2.51001408e-01],
       [9.58759652e-01, 4.12403476e-02],
       [9.24814051e-01, 7.51859490e-02],
       [9.64981812e-01, 3.50181879e-02],
       [9.65737903e-01, 3.42620973e-02],
       [3.85847522e-41, 1.00000000e+00],
       [5.97090856e-01, 4.02909144e-01],
       [9.64148007e-01, 3.58519927e-02],
       [2.86179073e-01, 7.13820927e-01],
       [9.66343404e-01, 3.36565964e-02]])

Calculate the score using cross validation

In [21]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.784     , 0.792     , 0.72      , 0.75806452, 0.80645161])